# Naive GAN training

In [3]:
from transformers import BartTokenizer, BartModel,BartForConditionalGeneration
from torch.distributions import Categorical
import pandas as pd
import torch.nn as nn
from undecorated import undecorated
import numpy as np
from datasets import load_metric
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset,DataLoader
from transformers import  Seq2SeqTrainingArguments, Seq2SeqTrainer,DataCollatorForSeq2Seq
metric = load_metric("rouge")
import torch.optim as optim
import torchvision.utils as vutils
import types
import matplotlib.pyplot as plt
from torch.distributions import Categorical
import random

<ipython-input-3-7dd2d3f9d1d8>:12: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("rouge")


In [4]:
from utils import PoemDataset,encode_sentences

In [17]:
from gan_utils import get_end_mask,get_valid_mask,Discriminator,discriminator_train_standard, generate_random_input,reinforce_loss,Generator



In [16]:
!python3 gan_utils.py

2022-12-17 19:41:52.171655: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-17 19:41:52.358909: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-17 19:41:52.961289: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64:/usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2022-12-17 19:41:52.961413: W tensorflow/stream_executor

In [6]:
tokenizer = BartTokenizer.from_pretrained('facebook/bart-base')

# Load Data

In [7]:
processed_data_dir =  '/home/alexander/nlg-project/data/'
test_df = pd.read_csv(processed_data_dir + 'test_data.csv')
test_model = encode_sentences(tokenizer,test_df)

train_df = pd.read_csv(processed_data_dir + 'train_data.csv')
train_model = encode_sentences(tokenizer,train_df)

In [8]:
train_ds = PoemDataset(train_model)
eval_ds = PoemDataset(test_model)

# Setup Training Params

In [11]:
PATH = '/home/alexander/nlg-project/GAN_models/'

In [9]:
 #reduced to a binary problem 
bsize = 12
generator =  Generator().cuda()

#uncomment if starting from checkpoint
#generator.load_state_dict(torch.load(PATH + 'generator.pt')['model_state_dict'])
discriminator = Discriminator().cuda()
#discriminator.load_state_dict(torch.load(PATH + 'discriminator.pt')['model_state_dict'])
criterion = nn.BCEWithLogitsLoss(reduction='none')

# Create batch of latent vectors that we will use to visualize

fixed_noise = generate_random_input(bsize,'cuda')


real_label = 1.
ewma_reward = 0.0
fake_label = 0.
lr = .000001
beta1 = .9
# Setup Adam optimizers for both G and D
optimizerD = optim.Adam(discriminator.parameters(), lr=lr*.05, betas=(beta1, 0.99)) 
optimizerG = optim.Adam(generator.parameters(), lr=lr, betas=(beta1, 0.99))



# TRAIN

In [12]:
# Training Loop
device = 'cuda'

# Lists to keep track of progress
img_list = []
G_losses = []
D_losses = []
iters = 0
num_epochs = 30

train_dl = DataLoader(train_ds,batch_size=bsize,num_workers=2)
print("Starting Training Loop...")
# For each epoch
for epoch in range(num_epochs):
    print("Label Noise:",0.05)
    # For each batch in the dataloader
    for i, data in enumerate(train_dl, 0):
       # print(i)
        
        if len(data['input_ids']) < bsize:
            continue
       # print(data)
        ############################
        # (1) Update D network: maximize log(D(x)) + log(1 - D(G(z)))
        ###########################
        ## Train with all-real batch
        
        # Format batch
        real_inputs = data['input_ids'].to(device)
        real_input_attention = data['attention_mask'].to(device)
        real_input_test = {'input_ids':real_inputs,'attention_mask':real_input_attention}
        
        
        real_gpu = data['label_ids'].to(device)
        attention_gpu = data['decoder_attention_mask'].to(device)
        b_size = real_gpu.size(0)
        #
        label = torch.full((b_size,), 1.0 , dtype=torch.float, device=device) #.95 instead of real to inject noise
        label = torch.bernoulli(label)
       # print(label)
        # Forward pass real batch through D
        real_mask = get_valid_mask(real_gpu)

        output_real,loss_real = discriminator_train_standard(discriminator,real_gpu[:,1:],label,criterion,mask=real_mask)

        loss_real.backward()

        #del label
        #output = discriminator(real_gpu,attention_mask=attention_gpu).view(-1)
        # Calculate loss on all-real batch
        #errD_real = criterion(output, label)
        # Calculate gradients for D in backward pass
        #errD_real.backward()
        D_x = output_real.detach().mean().item()
        #del output_real
        ## Train with all-fake batch
        # Generate batch of latent vectors
        noise = generate_random_input(bsize,device)#torch.randn(b_size, nz, 1, 1, device=device)
        # Generate fake image batch with G
        #print(fake)
       # print(noise)
        if random.random() > .5:

            fake,fake_logprobs = generator(noise)
        else:

            #sometimes the discirminator learns words in real input, this way some will be based on these same words
            fake,fake_logprobs = generator(real_input_test)
            #real_input_test
  
        
        fake_mask = get_valid_mask(fake)
        end_mask = get_end_mask(fake)
  
        #generate_sequence(generator,noise)
       # print(len(fake_logprobs))
        #a = 1/0
        #fake = generator.generate_with_grad(noise['input_ids'].cuda(),attention_mask=noise["attention_mask"].cuda()) #,max_length=128
       # print(fake)
        #print((fake.logits))
        #fake = decode(fake.logits)
      #  print(fake)
        # I am injecting some false positives into the fake labels so the discriminator does a worse job
        label.fill_(0.00) 
        label = torch.bernoulli(label)
       # print(label)
        # Classify all fake batch with D
        output_fake,loss_fake = discriminator_train_standard(discriminator,fake,label,criterion,mask=fake_mask)

        loss_fake.backward()
       # print('fake')
       # print(loss_fake.detach())
        #output = discriminator(fake).view(-1)
        # Calculate D's loss on the all-fake batch
        #errD_fake = criterion(output, label)
        # Calculate the gradients for this batch, accumulated (summed) with previous gradients
       # errD_fake.backward()
        D_G_z1 = output_fake.detach().cpu().mean().item()
        # Compute error of D as sum over the fake and the real batches
        errD = (torch.mean(loss_real.detach()) + torch.mean(loss_fake.detach())).cpu()
       # del output_real
        # Update D
        optimizerD.step()
        

        ############################
        # (2) Update G network
        ###########################
        
        #generator.zero_grad()

        lossG, cumulative_rewards, ewma_reward = reinforce_loss(output_fake,fake_logprobs,.3,.08,epoch,fake_mask,end_mask,ewma_reward)
        if i % 512 == 0:
            #for debugging
            print('Disc LogProbs True')
            print(output_real[0:1])
            print('Disc LogProbs False')
            print(output_fake[0:1])
            print('gen loss')
            print(lossG[0:1])
            print('cum rewards')
            print(cumulative_rewards[0:1])
            print('gen logprobs')
            print(fake_logprobs[0:1])
            print(tokenizer.batch_decode(fake[0:1].cpu()))

        #print(lossG)
        lossG = torch.mean(lossG)

        lossG.backward()
        D_G_z2 = lossG.detach().cpu().mean().item()
        # Update G
        
        optimizerG.step()
        generator.zero_grad()
        discriminator.zero_grad()

        # Output training stats
        if i % 256 == 0:
            print('[%d/%d][%d/%d]\tLoss_D: %.4f\tLoss_G: %.4f\tD(x): %.4f\tD(G(z)): %.4f / %.4f'
                  % (epoch, num_epochs, i, len(train_dl),
                     errD.item(), lossG.item(), D_x, D_G_z1, D_G_z2))
          #  print('ewma_reward:',ewma_reward)

        # Save Losses for plotting later
        G_losses.append(lossG.detach().item())
        D_losses.append(errD.detach().item())
        
        del lossG
        del errD

        # Check how the generator is doing by printing G's output on fixed_noise
        if (iters % 500 == 0) or ((epoch == num_epochs-1) and (i == len(dataloader)-1)):
            with torch.no_grad():
                fake = tokenizer.batch_decode(generator.model.generate(fixed_noise['input_ids'],attention_mask=fixed_noise["attention_mask"],max_length=32,num_beams= 50,early_stopping = True))
                print(fake[0:2])
        if (iters % 1000 == 0 ):
            print('saving geneartor')
            torch.save({
            'epoch': 15,
            'model_state_dict': generator.state_dict(),
            'optimizer_state_dict': optimizerG.state_dict(),
            'loss': 0.3355,
            }, PATH + 'generator.pt')
            torch.save({
            'epoch': 15,
            'model_state_dict': discriminator.state_dict(),
            'optimizer_state_dict': optimizerD.state_dict(),
            'loss': 0.3355,
            }, PATH + 'discriminator.pt')
            #img_list.append(vutils.make_grid(fake, padding=2, normalize=True))

        iters += 1

Starting Training Loop...
Label Noise: 0.05
Disc LogProbs True
tensor([[ 0.0143, -0.0546,  0.0058,  0.0117, -0.0504, -0.0379,  0.0013,  0.0016,
         -0.0111,  0.0134,  0.0088,  0.0211,  0.0421,  0.0366,  0.0471,  0.0383,
          0.0408,  0.0322,  0.0266,  0.0190,  0.0194]], device='cuda:0',
       grad_fn=<SliceBackward>)
Disc LogProbs False
tensor([[0.0063, 0.1021, 0.0927, 0.0983, 0.0985, 0.0747, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000]], device='cuda:0', grad_fn=<SliceBackward>)
gen loss
tensor([[ 9.6255e-07,  9.8022e-02,  5.6093e-02,  2.2434e-01,  2.0876e-01,
         -2.6792e-02,  0.0000e+00,  0.0000e+00,  0.0000e+00,  0.0000e+00,
          0.0000e+00]], device='cuda:0', grad_fn=<SliceBackward>)
cum rewards
tensor([[ 0.0032,  0.0663,  0.0644,  0.0696,  0.0702, -0.1134,  0.0000,  0.0000,
          0.0000,  0.0000,  0.0000]], device='cuda:0', grad_fn=<SliceBackward>)
gen logprobs
tensor([[-1.9073e-04, -2.0657e-03, -8.4617e-01, -3.1396e+00, -2.8973e+00,
         -2.4029e

Traceback (most recent call last):
  File "/home/alexander/anaconda3/lib/python3.8/multiprocessing/queues.py", line 245, in _feed
    send_bytes(obj)
  File "/home/alexander/anaconda3/lib/python3.8/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/alexander/anaconda3/lib/python3.8/multiprocessing/connection.py", line 411, in _send_bytes
    self._send(header + buf)
Traceback (most recent call last):
  File "/home/alexander/anaconda3/lib/python3.8/multiprocessing/queues.py", line 245, in _feed
    send_bytes(obj)
  File "/home/alexander/anaconda3/lib/python3.8/multiprocessing/connection.py", line 200, in send_bytes
    self._send_bytes(m[offset:offset + size])
  File "/home/alexander/anaconda3/lib/python3.8/multiprocessing/connection.py", line 411, in _send_bytes
    self._send(header + buf)
  File "/home/alexander/anaconda3/lib/python3.8/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)


RuntimeError: [enforce fail at inline_container.cc:262] . unexpected pos 473610112 vs 473610000

  File "/home/alexander/anaconda3/lib/python3.8/multiprocessing/connection.py", line 368, in _send
    n = write(self._handle, buf)
BrokenPipeError: [Errno 32] Broken pipe
